In [1]:
import datetime

import pandas as pd
import nltk
import torch

from blurr.text.data.all import *
from blurr.text.modeling.all import *
from fastai.data.block import RandomSplitter
from transformers import BartForConditionalGeneration

nltk.download("punkt")

/home/team_007/mambaforge/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/team_007/mambaforge/lib/python3.10/site-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
[nltk_data] Downloading package punkt to /home/team_007/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Grab our topics and transcripts

In [2]:
sheets_d = pd.read_excel(
    "../../data/raw/fsdl_2022_project_transcripts.xlsx", sheet_name=["lesson_topics", "lesson_transcripts"]
)
topics_df, transcripts_df = [v for k, v in sheets_d.items()]

topics_df.drop(columns="video_url", inplace=True)
transcripts_df.drop(columns="video_url", inplace=True)

topics_df["timestamp"] = topics_df["timestamp"].astype(str)
transcripts_df["timestamp"] = transcripts_df["timestamp"].astype(str)

In [3]:
print(len(topics_df))

topics_df.head()

397


,course_title,lesson_num,timestamp,topic
0,fast.ai 2022 - Part 1,2,00:00:00,Introduction
1,fast.ai 2022 - Part 1,2,00:00:55,Reminder to use the fastai book as a companion to the course
2,fast.ai 2022 - Part 1,2,00:02:06,aiquizzes.com for quizzes on the book
3,fast.ai 2022 - Part 1,2,00:02:36,"Reminder to use fastai forums for links, notebooks, questions, etc."
4,fast.ai 2022 - Part 1,2,00:03:42,How to efficiently read the forum with summarizations


In [4]:
print(len(transcripts_df))

transcripts_df.head()

14791


,course_title,lesson_num,timestamp,transcript
0,fast.ai 2022 - Part 1,2,00:00:00,"Hi everybody. Welcome to lesson two. Thanks for coming back… slight change of environment here,"
1,fast.ai 2022 - Part 1,2,00:00:08,we had a bit of an “administrative issue” at our university — somebody booked our room — so I'm
2,fast.ai 2022 - Part 1,2,00:00:14,doing this from the study at home. so sorry about the lack of decorations behind me.
3,fast.ai 2022 - Part 1,2,00:00:25,I'm actually really really pumped about this lesson. It feels like going back to what things
4,fast.ai 2022 - Part 1,2,00:00:32,"were like in the very early days, because we're doing some really new, really cool stuff, which…"


## Define a utility function for converting durations to total_seconds

In [5]:
def convert_duration_to_seconds(v):
    hrs, mins, secs = v.split(":")
    return (60 * 60 * int(hrs)) + (60 * int(mins)) + int(secs)

## Define the start/end boundaries (in seconds) for each topic in each lesson

In [6]:
topics_df["start_seconds"] = topics_df["timestamp"].apply(convert_duration_to_seconds)
topics_df["end_seconds"] = topics_df.groupby(by=["course_title", "lesson_num"])["start_seconds"].shift(
    -1, fill_value=100000
)

## Define the total number of elapsed seconds at each timestamp in the transcripts dataset

In [7]:
transcripts_df["elapsed_seconds"] = transcripts_df["timestamp"].apply(convert_duration_to_seconds)

In [8]:
transcripts_df.head()

,course_title,lesson_num,timestamp,transcript,elapsed_seconds
0,fast.ai 2022 - Part 1,2,00:00:00,"Hi everybody. Welcome to lesson two. Thanks for coming back… slight change of environment here,",0
1,fast.ai 2022 - Part 1,2,00:00:08,we had a bit of an “administrative issue” at our university — somebody booked our room — so I'm,8
2,fast.ai 2022 - Part 1,2,00:00:14,doing this from the study at home. so sorry about the lack of decorations behind me.,14
3,fast.ai 2022 - Part 1,2,00:00:25,I'm actually really really pumped about this lesson. It feels like going back to what things,25
4,fast.ai 2022 - Part 1,2,00:00:32,"were like in the very early days, because we're doing some really new, really cool stuff, which…",32


## Build our training data.  

This should be usable for both segmentation and summarization tasks

In [9]:
merged_df = topics_df[["course_title", "lesson_num", "topic", "start_seconds", "end_seconds"]].merge(
    transcripts_df, on=["course_title", "lesson_num"]
)
len(merged_df)

264993

Keep only the merged records where the transcript lies inbetween the start/end of the topic

In [10]:
merged_df = merged_df[
    (merged_df.elapsed_seconds >= merged_df.start_seconds) & (merged_df.elapsed_seconds < merged_df.end_seconds)
]

In [11]:
merged_df.head()

,course_title,lesson_num,topic,start_seconds,end_seconds,timestamp,transcript,elapsed_seconds
0,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:00,"Hi everybody. Welcome to lesson two. Thanks for coming back… slight change of environment here,",0
1,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:08,we had a bit of an “administrative issue” at our university — somebody booked our room — so I'm,8
2,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:14,doing this from the study at home. so sorry about the lack of decorations behind me.,14
3,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:25,I'm actually really really pumped about this lesson. It feels like going back to what things,25
4,fast.ai 2022 - Part 1,2,Introduction,0,55,00:00:32,"were like in the very early days, because we're doing some really new, really cool stuff, which…",32


For both segmentation and summarization tasks, we'll need to group the transcripts by course + lesson + topic

In [12]:
train_df = (
    merged_df[["course_title", "lesson_num", "topic", "transcript", "start_seconds"]]
    .groupby(by=["course_title", "lesson_num", "start_seconds", "topic"])
    .agg(list)
    .reset_index()
)

train_df.sort_values(by=["course_title", "lesson_num", "start_seconds"], inplace=True)

In [13]:
train_df.head()

,course_title,lesson_num,start_seconds,topic,transcript
0,Full Stack Deep Learning - Spring 2021,1,0,Intro,"[so josh talked about why we're doing, this course, and what's coming up this is the first, lecture, deep learning fundamentals that actually, has the i guess the content, but there's a lot of content in deep, learning fundamentals, and most of it is going to be a review, for most of you, that's what we assume that's the purpose, of our, requirements for enrolling but, if what i'm going to talk about in this, lecture real quick, is not going to be mostly review for you, then, i highly recommend that you go through, this uh online textbook, neural networks and deeplearning.com, it's, someth..."
1,Full Stack Deep Learning - Spring 2021,1,85,Neural Networks,"[so let's kick it off with neural, networks, um and you guys see the screen that has, like the slide and nothing else on it, right, okay so neural networks are called, neural because, they are biologically inspired, by neurons right which do all the, computing in our bodies, and the the kind of mental model of a, neuron, is that it's a cell that has, things coming out of the main part, called dendrites, and you can think of them as like, receptors of information, and then if enough stimulation has been, received by the dendrites, then the whole neuron does a thing, called firing, it's basi..."
2,Full Stack Deep Learning - Spring 2021,1,408,Universality,"[is universality which is, you know this neural network represents, some function y, right y equals f of of x the input, and then w the setting of all the, weights, but what can that function be right, let's look at this function on the left, f of x very, you know lots of peaks and valleys in, here, how can we know if there's a neural, network that, and a choice of weights for it that can, basically represent this function, and to summarize you know some, theoretical results, you can prove that any two layer neural, networks that's one hidden layer, right so inputs to one hidden layer two,..."
3,Full Stack Deep Learning - Spring 2021,1,528,Learning Problems,"[networks for, well we do for machine learning what, kind of machine learning, problems are there there's three kind of, big you know, a breakdown of all the machine learnings, out there you can have three categories, supervised learning unsupervised, learning and reinforcement learning, there's also transfer learning, meta learning you know imitation, learning all these, different types of learnings but these, are the three big categories, so supervised learning actually, unsupervised learning you get, unlabeled data x that means you know x, can be maybe, sound clips right or text like, t..."
4,Full Stack Deep Learning - Spring 2021,1,977,Empirical Risk Minimization / Loss Functions,"[um what's known as risk minimization and, the concept of loss functions, so let's talk about linear regression, for a second, um so linear regression is, the so here i'm showing you what's known, as one-dimensional data right so there's, one dimension on the x-axis there's some, number, and then there's another dimension on, the y-axis and that's the output so it's, one-dimensional input data producing, one-dimensional output and, the question we may want to ask is like, well if we get, an input let's say it's 30, how can we predict what the output is, likely to be, right given that all t..."


QA to training set

In [14]:
train_df[train_df["course_title"] == "fast.ai 2022 - Part 1"].iloc[0].transcript

["Welcome to Practical Deep Learning for coders, lesson one. This is version five of this course, and it's the first new one we've done in two years.",
 "So, we've got a lot of cool things to cover! It's amazing how much has changed.",
 'Here is an xkcd from the end of 2015.']

## Build segmentation training set

In [15]:
seg_train_df = train_df.copy()

In [16]:
seg_examples = []

for example_idx, example in seg_train_df.iterrows():
    for seq_idx, seq in enumerate(example["transcript"]):
        if len(example["transcript"]) > (seq_idx + 1):
            seg_examples.append(
                {
                    "course_title": example["course_title"],
                    "lesson_num": example["lesson_num"],
                    "topic": example["topic"],
                    "seq": str(seq),
                    "next_seq": str(example["transcript"][seq_idx + 1]),
                    "other_topic_seqs": [
                        str(txt) for i, txt in enumerate(example["transcript"]) if i != seq_idx and i != seq_idx + 1
                    ],
                }
            )

In [17]:
print(len(seg_examples))
print(seg_examples[0])

14394
{'course_title': 'Full Stack Deep Learning - Spring 2021', 'lesson_num': 1, 'topic': 'Intro', 'seq': "so josh talked about why we're doing", 'next_seq': 'this course', 'other_topic_seqs': ["and what's coming up this is the first", 'lecture', 'deep learning fundamentals that actually', 'has the i guess the content', "but there's a lot of content in deep", 'learning fundamentals', 'and most of it is going to be a review', 'for most of you', "that's what we assume that's the purpose", 'of our', 'requirements for enrolling but', "if what i'm going to talk about in this", 'lecture real quick', 'is not going to be mostly review for you', 'then', 'i highly recommend that you go through', 'this uh online textbook', 'neural networks and deeplearning.com', "it's", 'something you can probably do in like a', "focused day or two it's not", "like a big textbook it's more of a", 'website so i highly recommend that', 'and uh our weekly reading is actually', 'from this', "uh textbook it's a chapt

In [18]:
seg_train_df = pd.DataFrame(seg_examples)

In [19]:
print(len(seg_train_df))
seg_train_df.head()

14394


,course_title,lesson_num,topic,seq,next_seq,other_topic_seqs
0,Full Stack Deep Learning - Spring 2021,1,Intro,so josh talked about why we're doing,this course,"[and what's coming up this is the first, lecture, deep learning fundamentals that actually, has the i guess the content, but there's a lot of content in deep, learning fundamentals, and most of it is going to be a review, for most of you, that's what we assume that's the purpose, of our, requirements for enrolling but, if what i'm going to talk about in this, lecture real quick, is not going to be mostly review for you, then, i highly recommend that you go through, this uh online textbook, neural networks and deeplearning.com, it's, something you can probably do in like a, focused day or t..."
1,Full Stack Deep Learning - Spring 2021,1,Intro,this course,and what's coming up this is the first,"[so josh talked about why we're doing, lecture, deep learning fundamentals that actually, has the i guess the content, but there's a lot of content in deep, learning fundamentals, and most of it is going to be a review, for most of you, that's what we assume that's the purpose, of our, requirements for enrolling but, if what i'm going to talk about in this, lecture real quick, is not going to be mostly review for you, then, i highly recommend that you go through, this uh online textbook, neural networks and deeplearning.com, it's, something you can probably do in like a, focused day or two..."
2,Full Stack Deep Learning - Spring 2021,1,Intro,and what's coming up this is the first,lecture,"[so josh talked about why we're doing, this course, deep learning fundamentals that actually, has the i guess the content, but there's a lot of content in deep, learning fundamentals, and most of it is going to be a review, for most of you, that's what we assume that's the purpose, of our, requirements for enrolling but, if what i'm going to talk about in this, lecture real quick, is not going to be mostly review for you, then, i highly recommend that you go through, this uh online textbook, neural networks and deeplearning.com, it's, something you can probably do in like a, focused day or..."
3,Full Stack Deep Learning - Spring 2021,1,Intro,lecture,deep learning fundamentals that actually,"[so josh talked about why we're doing, this course, and what's coming up this is the first, has the i guess the content, but there's a lot of content in deep, learning fundamentals, and most of it is going to be a review, for most of you, that's what we assume that's the purpose, of our, requirements for enrolling but, if what i'm going to talk about in this, lecture real quick, is not going to be mostly review for you, then, i highly recommend that you go through, this uh online textbook, neural networks and deeplearning.com, it's, something you can probably do in like a, focused day or t..."
4,Full Stack Deep Learning - Spring 2021,1,Intro,deep learning fundamentals that actually,has the i guess the content,"[so josh talked about why we're doing, this course, and what's coming up this is the first, lecture, but there's a lot of content in deep, learning fundamentals, and most of it is going to be a review, for most of you, that's what we assume that's the purpose, of our, requirements for enrolling but, if what i'm going to talk about in this, lecture real quick, is not going to be mostly review for you, then, i highly recommend that you go through, this uh online textbook, neural networks and deeplearning.com, it's, something you can probably do in like a, focused day or two it's not, like a ..."


## Build summarization training set

In [20]:
summarization_train_df = train_df.copy()

In [21]:
summarization_train_df["transcript"] = summarization_train_df["transcript"].apply(
    lambda v: " ".join([str(seq) for seq in v])
)

In [22]:
summarization_train_df.head()

,course_title,lesson_num,start_seconds,topic,transcript
0,Full Stack Deep Learning - Spring 2021,1,0,Intro,so josh talked about why we're doing this course and what's coming up this is the first lecture deep learning fundamentals that actually has the i guess the content but there's a lot of content in deep learning fundamentals and most of it is going to be a review for most of you that's what we assume that's the purpose of our requirements for enrolling but if what i'm going to talk about in this lecture real quick is not going to be mostly review for you then i highly recommend that you go through this uh online textbook neural networks and deeplearning.com it's something you can probably d...
1,Full Stack Deep Learning - Spring 2021,1,85,Neural Networks,so let's kick it off with neural networks um and you guys see the screen that has like the slide and nothing else on it right okay so neural networks are called neural because they are biologically inspired by neurons right which do all the computing in our bodies and the the kind of mental model of a neuron is that it's a cell that has things coming out of the main part called dendrites and you can think of them as like receptors of information and then if enough stimulation has been received by the dendrites then the whole neuron does a thing called firing it's basically an electrical im...
2,Full Stack Deep Learning - Spring 2021,1,408,Universality,is universality which is you know this neural network represents some function y right y equals f of of x the input and then w the setting of all the weights but what can that function be right let's look at this function on the left f of x very you know lots of peaks and valleys in here how can we know if there's a neural network that and a choice of weights for it that can basically represent this function and to summarize you know some theoretical results you can prove that any two layer neural networks that's one hidden layer right so inputs to one hidden layer two outputs if given eno...
3,Full Stack Deep Learning - Spring 2021,1,528,Learning Problems,networks for well we do for machine learning what kind of machine learning problems are there there's three kind of big you know a breakdown of all the machine learnings out there you can have three categories supervised learning unsupervised learning and reinforcement learning there's also transfer learning meta learning you know imitation learning all these different types of learnings but these are the three big categories so supervised learning actually unsupervised learning you get unlabeled data x that means you know x can be maybe sound clips right or text like text strings but ther...
4,Full Stack Deep Learning - Spring 2021,1,977,Empirical Risk Minimization / Loss Functions,um what's known as risk minimization and the concept of loss functions so let's talk about linear regression for a second um so linear regression is the so here i'm showing you what's known as one-dimensional data right so there's one dimension on the x-axis there's some number and then there's another dimension on the y-axis and that's the output so it's one-dimensional input data producing one-dimensional output and the question we may want to ask is like well if we get an input let's say it's 30 how can we predict what the output is likely to be right given that all this data that we've...


In [23]:
summarization_train_df.iloc[0].transcript

"so josh talked about why we're doing this course and what's coming up this is the first lecture deep learning fundamentals that actually has the i guess the content but there's a lot of content in deep learning fundamentals and most of it is going to be a review for most of you that's what we assume that's the purpose of our requirements for enrolling but if what i'm going to talk about in this lecture real quick is not going to be mostly review for you then i highly recommend that you go through this uh online textbook neural networks and deeplearning.com it's something you can probably do in like a focused day or two it's not like a big textbook it's more of a website so i highly recommend that and uh our weekly reading is actually from this uh textbook it's a chapter from this book so today what we're going to talk about is neural networks universality of approximation the types of learning problems that neural networks can be used to solve loss functions and and minimizing them gr

## Blurr learner for training summarization model

In [24]:
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")

Using GPU #0: Tesla V100-SXM2-16GB


In [25]:
learn = BlearnerForSummarization.from_data(
    summarization_train_df,
    "sshleifer/distilbart-cnn-6-6",
    text_attr="transcript",
    summary_attr="topic",
    max_length=256,
    max_target_length=130,
    dblock_splitter=RandomSplitter(),
    dl_kwargs={"bs": 32},
).to_fp16()

/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [30]:
learn.fit_one_cycle(10, lr_max=4e-5, cbs=[BlearnerForSummarization.get_metrics_cb()])

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,rougeLsum,bertscore_precision,bertscore_recall,bertscore_f1,rouge1,rouge2,rougeL,rougeLsum,bertscore_precision,bertscore_recall,bertscore_f1,time
0,3.996448,3.738628,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.063692,0.012697,0.055451,0.061330,0.809005,0.865657,0.836259,00:18
1,3.780062,3.469175,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.068255,0.015796,0.058577,0.063986,0.810859,0.868150,0.838391,00:14
2,3.505078,3.326282,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.078303,0.021914,0.070290,0.073146,0.809613,0.871336,0.839143,00:13
3,3.194408,3.310852,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.073724,0.022267,0.064284,0.067749,0.805505,0.869810,0.836239,00:13
4,2.896657,3.416538,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.082064,0.023593,0.074779,0.078019,0.808542,0.871605,0.838720,00:13
5,2.616342,3.528751,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.072377,0.018873,0.065467,0.067861,0.798134,0.867135,0.830999,00:13
6,2.361720,3.633969,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.079125,0.023314,0.071311,0.075502,0.802681,0.868481,0.834081,00:13
7,2.149974,3.707175,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.076346,0.019703,0.069976,0.071863,0.800788,0.867932,0.832786,00:13
8,1.971459,3.737713,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.079448,0.017940,0.072168,0.073141,0.800576,0.867136,0.832307,00:13
9,1.831894,3.741242,0.070186,0.012283,0.058038,0.064639,0.808916,0.867189,0.836888,0.074097,0.016569,0.067867,0.069131,0.800944,0.866961,0.832428,00:13


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


...

/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3542: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Predictions and taking look at results

In [32]:
learn.show_results(learner=learn)

In [33]:
test_article = """hey everybody welcome back this week we're going to talk about something a little bit different than we do most weeks most weeks we talk about specific
technical aspects of building machine learning powered products but this week we're going to focus on some of the
organizational things that you need to do in order to work together on ml-powered products as part of an
interdisciplinary team so the the reality of building ml Power Products is that building any product well is really
difficult you have to figure out how to hire grade people you need to be able to manage those people and get the best out
of them you need to make sure that your team is all working together towards a shared goal you need to make good
long-term technical choices manage technical debt over time you need to make sure that you're managing
expectations not just of your own team but also of leadership of your organization and you need to be able to make sure
that you're working well within the confines of the requirements of the rest of the org that you're understanding
those requirements well and communicating back to your progress to the rest of the organization against those requirements
but machine learning adds even more additional complexity to this machine learning Talent tends to be very scarce
and expensive to attract machine learning teams are not just a
single role but today they tend to be pretty interdisciplinary which makes managing them an even bigger challenge
machine learning projects often have unclear timelines and there's a high
degree of uncertainty to those timelines machine learning itself is moving super fast and machine learning as we've
covered before you can think of as like the high interest credit card of technical debt so keeping up with making
good long-term decisions and not incurring too much technical debt is especially difficult in ml unlike
traditional software ml is so new that in most organizations leadership tends not to be that well educated in it they
might not understand some of the core differences between ML and other technology that you're working with machine learning products tend to fail
in ways that are really hard for Lay people to understand and so that makes it very difficult to help the rest of
the stakeholders in your organization understand what they could really expect from the technology that you're building
and what is realistic for us to achieve so throughout the rest rest of this lecture we're going to kind of touch on
some of these themes and cover different aspects of this problem of working together to build ml Power Products as
an organization so here are the pieces that we're going to cover we're going to talk about different roles that are involved in building ml products we're
going to talk about some of the unique aspects involved in hiring ml Talent
we're going to talk about organization of teams and how the ml team tends to fit into the rest of the org and some of
the pros and cons of different ways of setting that up we'll talk about managing ml teams and
ml product management and then lastly we'll talk about some of the design considerations for how to design a
product that is well suited to having a good ml model that backs it so let's dive in and talk about rules the most
common ml rules that you might hear of are things like ml product manager ml
"""

In [34]:
learn.predict(test_article, num_return_sequences=3)

/home/team_007/mambaforge/lib/python3.10/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'summary_texts': [' What To Hire (Organization/Supplier/Hire/Hole) The Top 5 ML Teams (Organs, or Not Top 10 ML Teams) Back to Mail Online home. Back to the page you came from.com/Top 5 ML Partners (Note: YOLO)',
   ' What To Hire (Organization/Supplier/Hire/Hole) The Top 5 ML Teams (Organs, or Not Top 10 ML Teams) Back to Mail Online home. Back to the page you came from.com/Top 5 ML Partners (UPS)',
   ' What To Hire (Organization/Supplier/Hire/Hole) The Top 5 ML Teams (Organs, or Not Top 10 ML Teams) Back to Mail Online home. Back to the page you came from.com/Locked/Top 5']}]